# Keep to Zettelkasten

Takes JSON files generated by Google Takeout, and converts them into Zettelkasten notes (with datetime in the beggining).

Then it saves them to folders by years.


In [ ]:
input_folder = 'Takeout/Google Keep'
output_folder = 'notes'

In [ ]:
import os
import glob
import json
from datetime import datetime as dt
import re



def copy_file(file, path, notespath):
    try:
        cp(f'{path}{file}', os.path.join(notespath, 'resources'))
    except FileNotFoundError:
        print(f'File "{file}" not found in {path}')
        return False
    else:
        return True

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)    #https://stackoverflow.com/a/58356570
    
# remove illegal chars for current OS
def clean_title(title) -> str:
    title = title.replace('/', '_')
    title = title.replace('\\', '_').replace('/', '_').replace('|', '_')
    title = title.replace('<', '-').replace('>', '-').replace(':', ' ')
    title = title.replace('?', '').replace('"', '').replace('*', '')
    title = title.replace('|', '').replace('«', '').replace('»', '')
    title = title.replace('—', '').replace('–', '').replace('×', '')
    title = title.replace("'", '').replace('’', '').replace('“', '')
    title = title.replace("₽", '').replace('•', '').replace('“', '')
    title = remove_emojis(title)
    title = title.replace('\n', '')
    return title

def read_annotations(list) -> str:
    annotations_list = '*Weblinks:*'
    for entry in list:
        if entry['source'] == 'WEBLINK':
            title = entry['title']
            url = entry['url']
            annotations_list += f' [{title}]({url});'
    return annotations_list

def format_tags(tags) -> str:
    tag_list = ''
    for tag in tags:
        tag_list += f' #{tag} '
    return tag_list

def read_write_notes(input_folder,output_folder):
    path = input_folder
    jsonpath = os.path.join(path, '')
    notes = glob.glob(f'{jsonpath}*.json')

    for note in notes:
        with open(note, 'r', encoding='utf-8') as jsonfile:
            data = json.load(jsonfile)
            timestamp = data['userEditedTimestampUsec']
            tags = []
            try:
                tags = [label['name'] for label in data['labels']]
            except KeyError:
                    print('No tags available.')

            if timestamp == 0:
                iso_datetime = dt.now().strftime('%Y%m%dT%H%M%S_edited')
            else:
                iso_datetime = dt.fromtimestamp(timestamp/1000000).strftime('%Y%m%d%H%M ')
            try:
                annotations = data['annotations']
            except KeyError:
                annotations = None
            # get filename by title
            if data['title'] != '':
                filename = str(data['title'])
            else:
                # put link if no title
                if annotations:
                    filename = annotations[0]['title']
                else:
                    filename = ''
            filename = clean_title(filename)
            if len(filename) > 50:
                filename = filename[0:49]        
            filename = iso_datetime + filename
            # create folders by years
            if len(iso_datetime)>0:
                subfolder = iso_datetime[:4]
            else:
                subfolder = ''
            notespath = os.path.join(output_folder, subfolder, '')

            # check if filename exists
            if os.path.exists(f'{notespath}{filename}.md'):
                print(f'Existing file found: {notespath}{filename}.md')
                # increment duplicate number, if already exists
                dup_num = 1
                while(os.path.exists(f'{notespath}{filename}({dup_num}).md')):
                    print(f'File "{notespath}{filename}({dup_num}).md" exists, increment number...')
                    dup_num = dup_num+1

                filename = f'{filename}({dup_num})'
                print(f'New filename: {filename}.md')

            # create path to notes
            if not os.path.exists(notespath):
                os.makedirs(notespath)
                print(f'Created year: {subfolder}')

            # create Markdown file
            print(f'Convert "{filename}" to markdown file.')
            with open(f'{notespath}{filename}.md', 'w', encoding='utf-8') as mdfile:

                if data['title'] != '':
                    mdfile.write(str(data['title']))
                mdfile.write(f'\n')
                mdfile.write(f'\n')
                # add text content
                try:
                    textContent = data['textContent']
                    mdfile.write(f'{textContent}\n\n')
                except KeyError:
                    print('No text content available.')
                if annotations:
                    annotations = read_annotations(annotations)
                    mdfile.write(f'{annotations}')
                mdfile.write(f'\n\n')
                mdfile.write(f'tags: ')
               # add tags
                if tags:
                    mdfile.write(f'{format_tags(tags)}')

                mdfile.write(f'#google_keep')

def create_folder():
    try:
        workpath = os.path.join(output_folder)
        if not os.path.exists(workpath):
            os.makedirs(workpath)
            print('Create folder "{}" - home of markdown files.'.format(output_folder))
    except OSError:
        print('Creation of folders failed.')




In [ ]:
create_folder()
try:
    read_write_notes(input_folder,output_folder)
except IndexError:
    print('Please enter a correct path!')
